In [ ]:
import numpy as np
from collections import defaultdict

from utils.microbench_parser import MicrobenchParser
from utils.plot import plot_with_models
from utils.model import OpTimeModelBase

In [ ]:
SWITCH_TIME = 50
NUM_PREFETCHES = 12
IO_COMPLETION_TIME = 140
IO_SUBMISSION_TIME = 1690 - IO_COMPLETION_TIME
PAUSE_TIME = 12

In [ ]:
mp = MicrobenchParser(['../microbench/log/dram.json',
                       '../microbench/log/cxl.json'])

In [ ]:
perfs = defaultdict(list)
model = OpTimeModelBase(SWITCH_TIME, NUM_PREFETCHES)
latency_list = mp.get_values('latency')
for num_chases in mp.get_values('num_chases'):
    # compute memory time offset from DRAM execution
    min_memory_time = mp.get_values('memory_time')[0]
    min_io_time_pre = mp.get_values('io_time_pre')[0]
    min_io_time_post = mp.get_values('io_time_post')[0]
    dram_time = 1e9 / mp.get_throughputs(latency_list, num_chases, min_memory_time, min_io_time_pre, min_io_time_post)[0]
    model_time = model.reciprocal_throughput(num_chases, PAUSE_TIME * min_memory_time,
                                             min_io_time_pre + IO_SUBMISSION_TIME, min_io_time_post + IO_COMPLETION_TIME, 0)
    memory_time_offset = (dram_time - model_time) / num_chases

    for memory_time in mp.get_values('memory_time'):
        for io_time_pre in mp.get_values('io_time_pre'):
            for io_time_post in mp.get_values('io_time_post'):
                throughputs = mp.get_throughputs(latency_list, num_chases, memory_time, io_time_pre, io_time_post)
                memory_time_nsec = PAUSE_TIME * memory_time + memory_time_offset
                io_time_pre_ofs = io_time_pre + IO_SUBMISSION_TIME
                io_time_post_ofs = io_time_post + IO_COMPLETION_TIME
                title = '$M$ = %d, $T_{\mathrm{mem}}$ = %d nsec, $T_{\mathrm{IO}}^{\mathrm{pre}}$ = %d nsec, $T_{\mathrm{IO}}^{\mathrm{post}}$ = %d nsec' % (num_chases, memory_time_nsec, io_time_pre_ofs, io_time_post_ofs)
                name = 'microbench_M%d_mem%d_IOpre%d_IOpost%d' % (num_chases, memory_time_nsec, io_time_pre_ofs, io_time_post_ofs)
                p = plot_with_models(latency_list, throughputs, num_chases, memory_time_nsec, io_time_pre_ofs, io_time_post_ofs,
                                     SWITCH_TIME, NUM_PREFETCHES, 'Microbench', title, name)
                for k, v in p.items():
                    perfs[k] += v

for k, v in perfs.items():
    if(k != 'Microbench'):
        ratios = list(np.array(v) / np.array(perfs['Microbench']))
        print('%s: %.1f %% overestimate, %.1f %% underestimate' % (k, (max(ratios) - 1) * 100, (1 - min(ratios)) * 100))